In [1]:
import pandas as pd
import numpy as np
import scipy.io
import pickle
import random
from collections import OrderedDict

In [2]:
mat = scipy.io.loadmat("/mnt/storage/data/vision/imagenet/2012/devkit_task_1_and_2/data/meta.mat")

In [3]:
rows = []
for i in range(1860):
    row_id = mat['synsets'][i][0][0][0][0]
    wnid = mat['synsets'][i][0][1][0]
    words = mat['synsets'][i][0][2][0]
    gloss = mat['synsets'][i][0][3][0]
    num_children = mat['synsets'][i][0][4][0][0]
    children = np.array2string(mat['synsets'][i][0][5][0], separator=",")[1:-1]
    wordnet_height = mat['synsets'][i][0][6][0][0]
    num_train_images = mat['synsets'][i][0][7][0][0]
    row = [row_id, wnid, words, gloss, num_children, children, wordnet_height, num_train_images]
    rows.append(row)
metadata_df = pd.DataFrame(columns=["ILSVRC2012_ID", "WNID", "words", "gloss", "num_children", "children", "wordnet_height", "num_train_images"], data=rows)

In [4]:
metadata_df

,ILSVRC2012_ID,WNID,words,gloss,num_children,children,wordnet_height,num_train_images
0,1,n02119789,"kit fox, Vulpes macrotis",small grey fox of southwestern United States; ...,0,,0,1300
1,2,n02100735,English setter,an English breed having a plumed tail and a so...,0,,0,1300
2,3,n02110185,Siberian husky,breed of sled dog developed in northeastern Si...,0,,0,1300
3,4,n02096294,Australian terrier,small greyish wire-haired breed of terrier fro...,0,,0,1300
4,5,n02102040,"English springer, English springer spaniel",a breed having typically a black-and-white coat,0,,0,1300
...,...,...,...,...,...,...,...,...
1855,1856,n06791372,"signal, signaling, sign",any nonverbal action or gesture that encodes a...,1,1857,7,0
1856,1857,n06873571,visual signal,a signal that involves visual communication,1,1858,3,0
1857,1858,n06874019,light,"a visual warning signal; ""they saw the light o...",1,861,2,0
1858,1859,n00031921,relation,an abstraction belonging to or characteristic ...,1,1860,14,0


In [68]:
metadata_df.to_pickle("./imagenet_2012_metadata_df.pkl")

In [4]:
img_count = pd.read_csv("id_to_img_count_mapping.txt", header=None)
labels = pd.read_csv("id_to_label_mapping.txt", header=None)
synset = pd.read_csv("id_to_synset_mapping.txt", header=None)

In [5]:
meta_df = pd.DataFrame({"labels": labels.values.flatten(), "synset": synset.values.flatten(), "img_count": img_count.values.flatten()})

In [6]:
meta_df.to_pickle("./train_image_metadata.pkl")

In [7]:
img_name_arr = []
img_count_arr = img_count.values.flatten()
for i in range(len(img_count_arr)):
    for c in range(img_count_arr[i]):
        img_name_arr.append(str(i).zfill(3) + "_" + str(c).zfill(4))

In [8]:
random.shuffle(img_name_arr)

In [9]:
img_to_shard_dict = {}
cur_shard_id = 0
cur_shard_cnt = 0
shard_size = 32768
for i in range(len(img_name_arr)):
    if cur_shard_cnt >= shard_size:
        cur_shard_id += 1
        cur_shard_cnt = 0
    img_to_shard_dict[img_name_arr[i]] = cur_shard_id
    cur_shard_cnt += 1

In [10]:
with open('img_to_shard_dict.pkl', 'wb') as handle:
    pickle.dump(img_to_shard_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
shard_to_imgs_dict = {}
for k, v in img_to_shard_dict.items():
    if v not in shard_to_imgs_dict:
        shard_to_imgs_dict[v] = [k]
    else:
        shard_to_imgs_dict[v].append(k)

In [12]:
with open('shard_to_imgs_dict.pkl', 'wb') as handle:
    pickle.dump(shard_to_imgs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
for k, v in shard_to_imgs_dict.items():
    shard_id = k
    imgs_arr = v
    shard_image_mapping_filepath = "/mnt/storage/data/vision/imagenet/2012/train_data_partioning/" + str(shard_id).zfill(3) + "_images.csv"
    shard_image_mapping_file = open(shard_image_mapping_filepath, "w+")
    shard_labels_filepath = "/mnt/storage/data/vision/imagenet/2012/train_data_partioning/" + str(shard_id).zfill(3) + "_labels.txt"
    shard_labels_file = open(shard_labels_filepath, "w+")
    for img_name in imgs_arr:
        img_name_split = img_name.split("_")
        img_class = img_name_split[0]
        img_num = img_name_split[1]
        row_off = random.randint(0, 31)
        col_off = random.randint(0, 31)
        line = img_class + "," + img_num + "," + str(row_off).zfill(2) + "," + str(col_off).zfill(2) + "\n"
        shard_image_mapping_file.write(line)
        shard_labels_file.write(img_class + "\n")
    shard_image_mapping_file.close()
    shard_labels_file.close()